<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Einführung in die Programmierung
### Winterersemester 2025/25
Prof. Dr. Stefan Goetze

# Übung X - Aufgabe 1 - Klassifikation von Zahlen mit einem CNN

## 1. Nötige Imports vornehmen

Die folgende Code-Zellen überprüfen zuerst, ob PyTorch installiert ist und importieren dann die notwendigen Bibliotheken, die im Folgenden benötigt werden.

In [ ]:
# Check if PyTorch is already installed
try:
    import torch
    print("PyTorch is already installed.")
except ImportError:
    print("PyTorch is not installed. Installing now...")
    %conda install pytorch torchvision -c pytorch

In [ ]:
import torch
import torch.nn as nn
#import torchvision
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
#import torchvision.models as models
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
#import numpy as np
#import pickle  # oder alternativ json

## 2. Trainingsdaten laden

Zunächst werden die Trainingsdaten geladen. Dabei wollen wir den [MNIST-Zahlen Datenset][MNIST] verwenden. Die Funktion *[toTensor()][PyTorchToTensor]* skaliert die Bilder automatisch nach `[0.0, 1.0]`. Daher ist eine manuelle Skalierung nicht nötig.

Die Skalierung ist wichtig, weil neuronale Netzwerke empfindlich auf den Wertebereich der Eingabedaten reagieren. Eingabewerte im Bereich `[0.0, 1.0]` sorgen für eine stabilere und schnellere Konvergenz beim Training. Würden wir z. B. die rohen Pixelwerte von 0 bis 255 verwenden, könnten Gradienten zu groß oder zu klein werden, was das Training erschwert oder verlangsamt.

[PyTorchToTensor]: https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.ToTensor
[MNIST]: https://de.wikipedia.org/wiki/MNIST-Datenbank

In [ ]:
mnist_transform = transforms.Compose([
    transforms.ToTensor(), 
])

# YOUR CODE HERE
raise NotImplementedError()

Schaun wir uns an, aus wie vielen Datenpunkte unser Datensatz besteht

In [ ]:
train_set, valid_set

Wir können uns auch anschauen, im welchen Format die Daten vorliegen. Dies wird später für das CNN wichtig.

In [ ]:
train_set[1][0].shape

### 2.1 Anzeigen der Bilder aus dem Datensatz

Statt `inp[0]` kann auch die Funktion *[squeeze()][2]* verwendet werden. Wie wir vorher gesehen haben, haben die Bilddaten die Form `[1, 28, 28]`, wobei die erste Dimension den Kanal (in diesem Fall Graustufen) repräsentiert.

Da `plt.imshow()` jedoch ein 2D-Array erwartet (Form `[H, W]`), muss die Kanaldimension entfernt werden. Sowohl `inp[0]` als auch `torch.squeeze(inp)` liefern das gewünschte Format `[28, 28]`.

[2]: https://pytorch.org/docs/main/generated/torch.squeeze.html


In [ ]:
def imshow(inp):
    plt.imshow(inp[0], cmap='gray')  # Graustufenbild anzeigen
    plt.axis('off')  # Achsen ausblenden
    plt.show()

Die folgende Zelle zeigen 50 Beispielbilder aus dem Testdatensatz, indem aus jedem Bild der erste (und einzige) Kanal `image[0]` extrahiert und als Graustufenbild mit dem zugehörigen Label dargestellt wird.


In [ ]:
fig, axes = plt.subplots(5, 10, figsize=(12, 7), subplot_kw={'xticks': [], 'yticks': []})

# Zeige einige Bilder aus dem Trainingsdatensatz an
for i, ax in enumerate(axes.flat):
    image, label = valid_set[i]
    ax.imshow(image[0], cmap=plt.cm.gray_r)  
    ax.set_title('label ' +str(i) + ': ' + str(label), fontsize=8)

# add title
plt.suptitle('MNIST Dataset Samples', fontsize=16)
plt.tight_layout()
plt.show()

Beispielbild aus dem Trainingsdatensatz anzeigen mit Label

In [ ]:
imshow(train_set[500][0])
print(f"Label: {valid_set[500][1]}")  # Das zugehörige Label anzeigen

## 3. Convolutional Neural Network erstellen

Das CNN basiert auf der Eingabegröße `[1, 28, 28]` der MNIST-Bilder. Die erste Convolution-Schicht verwendet einen 3×3-Kernel mit Padding=1. Padding sorgt dafür, dass die räumliche Auflösung nach der Faltung erhalten bleibt (28×28).  

Nach jedem Convolution-Schritt folgt ein MaxPooling mit einem 2×2-Fenster, das Höhe und Breite halbiert.

Die Form der Feature Maps verändert sich dabei wie folgt:
- Nach der ersten Conv- und Pooling-Schicht: `(32, 14, 14)`
- Nach der zweiten Conv- und Pooling-Schicht: `(64, 7, 7)`

Bevor die Daten in die Fully Connected Layer gehen, werden sie flach gemacht. Deshalb muss die Eingabegröße für `fc1` genau `64 * 7 * 7` sein, also 3136.


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)   # (1, 28, 28) -> (32, 28, 28)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # (32, 14, 14) -> (64, 14, 14)
        self.pool = nn.MaxPool2d(2, 2)                            # halbiert Auflösung

        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)  # 10 Klassen für Ziffern 0–9

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))   # (32, 28, 28) -> (32, 14, 14)
        x = self.pool(F.relu(self.conv2(x)))   # (64, 14, 14) -> (64, 7, 7)
        x = x.view(-1, 64 * 7 * 7)             # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


## 4. Trainieren des CNNs

Wir trainieren unser CNN-Modell und visualisieren anschließend die Ergebnisse. Wie wir sehen, brauchen wir tatsächlich nicht viele Epochen – bereits nach **3 Epochen** erreichen wir eine hohe Genauigkeit.

Das liegt daran, dass der **MNIST-Datensatz** relativ einfach ist:  
Er besteht aus kleinen, zentrierten Graustufenbildern von handgeschriebenen Ziffern (28x28 Pixel), die sich gut voneinander unterscheiden. Ein CNN kann durch seine Fähigkeit zur automatischen Merkmalsextraktion (z. B. durch Faltung und Pooling) schnell lernen, worauf es bei den Ziffern ankommt.



In [ ]:
def plot_results(epochs, training_acc, testing_acc, training_loss, testing_loss):
    plt.plot(range(epochs), training_acc, label="train_acc")
    plt.plot(range(epochs), testing_acc, label="valid_acc")
    plt.legend()
    plt.xlabel("epoch")
    plt.ylabel("accuracy")
    plt.show()

    plt.plot(range(epochs), training_loss, label="train_loss")
    plt.plot(range(epochs), testing_loss, label="valid_loss")
    plt.legend()
    plt.xlabel("epoch")
    plt.ylabel("loss")
    plt.show()

def train_model(model, epochs=3, learning_rate=0.001):
    # Hier Initialisierung der Verlustfunktion und des Optimierers
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    training_loss = []
    testing_loss = []
    training_acc = []
    testing_acc = []

    if torch.cuda.is_available():
        model = model.to("cuda")

    device = torch.device("cuda")
    print(device)
    
    # Training des Modells
    with tqdm(range(epochs)) as iterator:
        for epoch in iterator:
            train_loss = 0
            train_acc = 0

            model.train()
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)

                optimizer.zero_grad()
                output = model(images)
                loss = loss_fn(output, labels)

                loss.backward()
                optimizer.step()

                prediction = torch.argmax(output, dim=1)
                train_acc += (prediction == labels).sum().item()
                train_loss += loss.item()

            training_acc.append(train_acc/len(train_set))
            training_loss.append(train_loss/len(train_set))

            # Evaluation des Modells auf den Testdaten
            test_loss = 0
            test_acc = 0
            with torch.no_grad():
                for images, labels in valid_loader:
                    images, labels = images.to(device), labels.to(device)
                    output = model(images)
                    loss = loss_fn(output, labels)
                    prediction = torch.argmax(output, dim=1)

                    test_acc += (prediction == labels).sum().item()
                    test_loss += loss.item()

                testing_acc.append(test_acc/len(valid_set))
                testing_loss.append(test_loss/len(valid_set))

            iterator.set_postfix_str(f"train_acc: {train_acc/len(train_set):.2f} test_acc: {test_acc/len(valid_set):.2f} train_loss: {train_loss/len(train_set):.2f} test_loss: {test_loss/len(valid_set):.2f}")

    # Modell speichern
    torch.save(model.state_dict(), 'model_weights.pth')
    
    # Speichere die Metriken in einer Datei
    metrics = {
        'training_acc': training_acc,
        'testing_acc': testing_acc,
        'training_loss': training_loss,
        'testing_loss': testing_loss
    }
    
    #with open('training_metrics.pkl', 'wb') as f:
     #   pickle.dump(metrics, f)

    plot_results(epochs, training_acc, testing_acc, training_loss, testing_loss)



In [ ]:
model = CNN()
train_model(model)

Wir können raus lesen, dass unser Modell am Ende eine Accuracy von 99,2% hat

Schauen wir uns ein Test image an

In [ ]:
# Modell in Evaluationsmodus
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Ein Bild aus dem Testdatensatz holen
image, label = valid_set[22]  # z.B. zufälliges Bild
image = image.unsqueeze(0).to(device)  # Batch-Dimension hinzufügen: (1, 1, 28, 28)

# Vorhersage machen
with torch.no_grad():
    output = model(image)
    pred = torch.argmax(output, dim=1).item()

# Bild anzeigen mit Prediction
plt.imshow(image.cpu().squeeze(), cmap='gray')
plt.title(f"Prediction: {pred} | Label: {label}")
plt.axis('off')
plt.show()
